In [ ]:
using LinearAlgebra
using SparseArrays

In [ ]:
# let's define a shorthand for kron
const ⊗ = kron

# Getting started
-----

## Review
- Gradient descent
- Newton's method and KKT conditions
- Regularization
- Newton approximations
- Line search
- Exercises with GRAPE 

## Goals
- Introduce trajectory optimization
- Solve the LQR problem three ways
- Describe nonlinear trajectory optimization

## Lessons from exercises

### GRAPE
- Does not utilize temporal structure.
\begin{align}
\min_{u_{1:N}} &\quad J(u_{1:N}) 
\end{align}

### GRAPE: First order

- Slow

### GRAPE: Second order
- Fast

# I. Trajectory optimization
-----

\begin{align}
\min_{x_{1:N}, u_{1:N}} &\quad J(x_{1:N}, u_{1:N}) = \sum_{n=1}^N \ell_n(x_n, u_n) + \ell_N(x_N, u_N) \\
\text{s.t.} &\quad x_{n+1} = f(x_n, u_n, n)
\end{align}

# II. Linear Quadratic Regulator
-----

- LQR is the "simple Harmonic oscillator" of control

\begin{align}
\min_{x_{1:N}, u_{1:N}} &\quad J = \sum_{n=1}^N \tfrac{1}{2} x_n^T Q_n x_n + \tfrac{1}{2} u_n^T R_n u_n + \tfrac{1}{2} x_N^T Q_N x_N \\
\text{s.t.} &\quad x_{n+1} = A_n x_n + B_n u_n \\
&\quad Q_n \succeq 0,\, R_n \succ 0
\end{align}

## Linear system

### Zero-order hold

- Zero-order hold can be used to convert continuous, linear, time-invariant (LTI) systems to discrete LTI systems.

\begin{equation}
\dot{x} = A x + B u \overset{h}{\longrightarrow} \overset{+}{x} = A_h x + B_h u = \left( \sum_{n\ge0} \tfrac{1}{n!} A^n h^n \right) x + \left( \sum_{n\ge1} \tfrac{1}{n!} A^{n-1} B h^n \right) u \approx (I + h A) x + h B u
\end{equation}

- Matrix exponential trick:

\begin{equation}

\exp\left(\begin{bmatrix} A & B \\ 0 & 0 \end{bmatrix} h \right)
= \begin{bmatrix} A_h & B_h \\ 0 & I \end{bmatrix}

\end{equation}

In [ ]:
# Define continuous LTI system matrices
A = [0.0 1.0; -1.0 -0.1]
B = [0.0; 1.0]
h = 0.1  # Time step

function continuous_to_discrete(A, B, h)
    # Construct augmented matrix for matrix exponential
    augmented_matrix = [
        A B; 
        zeros(size(B, 2), size(A, 1)) zeros(size(B, 2), size(B, 2))
    ]

    # Compute matrix exponential
    exp_matrix = exp(augmented_matrix * h)

    # Extract discrete LTI system matrices
    A_h = exp_matrix[1:size(A, 1), 1:size(A, 2)]
    B_h = exp_matrix[1:size(A, 1), size(A, 2)+1:end]

    return A_h, B_h
end

# Extract discrete LTI system matrices
A_h, B_h = continuous_to_discrete(A, B, h);

In [ ]:
A_h

In [ ]:
I + A * h

In [ ]:
B_h

In [ ]:
B * h

### Double Integrator

- Double integrator (Newton's second law, $F = ma$)

\begin{equation}

m \frac{d}{dt} \begin{bmatrix} q \\ \dot{q} \end{bmatrix} 
= \begin{bmatrix} 0 & m \\ 0 & 0 \end{bmatrix} \begin{bmatrix} q \\ \dot{q} \end{bmatrix}
+ \begin{bmatrix} 0 \\ u \end{bmatrix}

\end{equation}

In [ ]:
function double_integrator(m)
    A_c = [0.0 1.0; 0.0 0.0]
    B_c = [0.0; 1.0 / m]
    return A_c, B_c
end

In [ ]:
m = 2.0 # Mass
A_c , B_c = double_integrator(m)
h = 0.05  # Time step
A, B = continuous_to_discrete(A_c, B_c, h);

In [ ]:
A ≈ I + A_c * h

In [ ]:
B ≈ B_c * h + [h^2 / 2m; 0]

## Indirect optimal control: Naive way

- Indirect optimal control is also known as "single shooting"
- The naive way is to perform gradient descent without any problem structure, like in GRAPE.

\begin{align}
\min_{u_{1:N}} &\quad J(u_{1:N}) = \sum_{n=1}^N \ell_n(x_n(u_{1:n{-}1}), u_n) + \ell_N(x_N(u_{1:N{-}1}), u_N) \\
\end{align}

- We will start with the double integrator and solve the LQR problem,
\begin{align}
\min_{u_{1:N}} &\quad J = \sum_{n=1}^N \tfrac{1}{2} x_n(u_{1:n{-}1})^T Q_n x_n(u_{1:n{-}1}) + \tfrac{1}{2} u_n^T R_n u_n + \tfrac{1}{2} x_N(u_{1:N{-}1})^T Q_N x_N(u_{1:N{-}1}) \\
\text{s.t.} &\quad Q_n \succeq 0,\, R_n \succ 0
\end{align}


In [ ]:
m = 2.0 # Mass
A_c , B_c = double_integrator(m)
h = 0.05  # Time step
A, B = continuous_to_discrete(A_c, B_c, h);

In [ ]:
function J(u; x₁=[0; 0], Q = 1.0I, R = 0.01I, QN = 100.0I)
    N = length(u) + 1
    J = 0.0
    xₙ = x₁
    for n in 1:N-1
        uₙ = u[n]
        J += 0.5 * (xₙ' * Q * xₙ + uₙ' * R * uₙ)
        xₙ = A * xₙ + B * uₙ
    end
    J += 0.5 * (xₙ' * QN * xₙ)
    return J
end

### Gradient descent

### Newton's method

## Indirect optimal control: Apply KKT

- Use KKT conditions because we have a constraint
- We can this in a smart way using the temporal structure of the problem

\begin{align}
\min_{x_{1:N}, u_{1:N}} &\quad J = \sum_{n=1}^N \tfrac{1}{2} x_n^T Q_n x_n + \tfrac{1}{2} u_n^T R_n u_n + \tfrac{1}{2} x_N^T Q_N x_N \\
\text{s.t.} &\quad x_{n+1} = A_n x_n + B_n u_n \\
&\quad Q_n \succeq 0,\, R_n \succ 0
\end{align}



In [ ]:
m = 2.0 # Mass
A_c , B_c = double_integrator(m)
h = 0.05  # Time step
A, B = continuous_to_discrete(A_c, B_c, h);

## Direct optimal control

- Package the optimization variables into a trajectory
- Set up a quadratic program defined over the trajectory
- Observe the presence of sparsity

\begin{align}
\min_{x_{1:N}, u_{1:N}} &\quad J = \sum_{n=1}^N \tfrac{1}{2} x_n^T Q_n x_n + \tfrac{1}{2} u_n^T R_n u_n + \tfrac{1}{2} x_N^T Q_N x_N \\
\text{s.t.} &\quad x_{n+1} = A_n x_n + B_n u_n \\
&\quad Q_n \succeq 0,\, R_n \succ 0
\end{align}

In [ ]:
m = 2.0 # Mass
A_c , B_c = double_integrator(m)
h = 0.05  # Time step
A, B = continuous_to_discrete(A_c, B_c, h);

N = 100
x1 = [1.0; 0.0] # Initial state

# Use sparse arrays
Q = sparse(1.0I(2))
R = sparse(0.01I(1))
QN = sparse(10.0I(2))

Define 
\begin{align}
Z &= \begin{bmatrix} 
    x_1 & x_2 & \cdots & x_N \\
    u_1 & u_2 & \cdots & u_N 
\end{bmatrix} \\
\Rightarrow \vec{Z} &= \begin{bmatrix} x_1 \\ u_1 \\ x_2 \\ u_2 \\ \vdots \\ x_N \\ u_N \end{bmatrix}
\end{align}
and drop the first state (known) and last control (not used),
\begin{equation}
 z = \vec{Z}[\text{length}(x_1){:}\text{end}{-}\text{length}(u_N)]
\end{equation}

Write the cost function as $J = \tfrac{1}{2} z^T H z$, where
\begin{equation}
    H = \begin{bmatrix}
        R_1 & 0 & 0 & & 0 \\
        0 & Q_2 & 0 & \cdots & 0 \\
        0 & 0 & R_2 & & 0 \\
        & \vdots & & \ddots & \vdots \\
        0 & 0 & 0 & \cdots & Q_N \\
    \end{bmatrix}.
\end{equation}

In [ ]:
# Recall our shorthand for kron
H = blockdiag(R, I(N-2) ⊗ blockdiag(Q, R), QN)

Write the dynamics constraint, $Cz = d$, where
\begin{equation}
C = \begin{bmatrix}
    -B_1 & I & 0 & 0 & & 0 \\
    0 & -A_2 & -B_2 & I & \cdots & 0 \\
    \vdots & \vdots & \ddots & \ddots & \ddots & 0 \\
    0 & 0 & \cdots & -A_{N-1} & -B_{N-1} & I
\end{bmatrix}, \qquad
d = \begin{bmatrix}
    A_1 x_1 \\
    0 \\
    0 \\
    \vdots \\
    0
\end{bmatrix}
\end{equation}


In [ ]:
C = [-B I(2)]

In [ ]:
d = [-A * x1; zeros(size(C, 1) - length(x1)]

Putting it all together, 
\begin{align}
    \min_z &\quad \tfrac{1}{2} z^T H z \\
    \text{s.t.}&\quad C z = d
\end{align}

- Lagrangian:
\begin{equation}
    L(z, \lambda) = \tfrac{1}{2} z^T H z + \lambda^T (C z - d)
\end{equation}

- KKT conditions:
\begin{align}
    & \nabla_z L = H z + C^T \lambda \overset{!}{=} 0 \\
    & \nabla_\lambda L = Cz - d \overset{!}{=} 0 \\
\end{align}

- Matrix form:
\begin{equation}
    \Rightarrow \begin{bmatrix} H & C^T \\ C & 0 \end{bmatrix} 
    \begin{bmatrix} z \\ \lambda \end{bmatrix} 
    = \begin{bmatrix} 0 \\ d \end{bmatrix}
\end{equation}

- How many iterations will this take to solve?

# III. Any last topics?
-----

- could teach DARE
- could mention nonlinear or leave for exercises